# Microtask 2



In [5]:
import json
import csv
import datetime
from collections import defaultdict
import pandas as pd

In [6]:
# Here both start year and end year are included
start_year = 2017
end_year = 2018 

quar1_dates = {"start_date": "01-01", "end_date": "03-31"}
quar2_dates = {"start_date": "04-01", "end_date": "06-30"}
quar3_dates = {"start_date": "07-01", "end_date": "09-30"}
quar4_dates = {"start_date": "10-01", "end_date": "12-31"}

old_committers = set()
old_issue_subs = set()
old_pr_subs = set()

In [7]:
class Quarter:
    
    def __init__(self, number, year):
        self.number = number    #.
        self.year = year   
   
        self.num_commits = 0  
        self.num_issues = 0
        self.num_pullrequests = 0
        
        self.new_committers = 0
        self.new_issue_subs = 0
        self.new_pr_subs = 0
        
        self.start_date = ""
        self.end_date = ""
        
        if self.number == 1:
            self.start_date = str(self.year) + '-' + quar1_dates["start_date"]
            self.end_date = str(self.year) + '-' + quar1_dates["end_date"]
            
        if self.number == 2:
            self.start_date = str(self.year) + '-' + quar2_dates["start_date"]
            self.end_date = str(self.year) + '-' + quar3_dates["end_date"]            
            
        if self.number == 3:
            self.start_date = str(self.year) + '-' + quar3_dates["start_date"]
            self.end_date = str(self.year) + '-' + quar3_dates["end_date"]
            
        if self.number == 4:
            self.start_date = str(self.year) + '-' + quar4_dates["start_date"]
            self.end_date = str(self.year) + '-' + quar4_dates["end_date"]
        
            
    def is_includes_data(self, date):
        if self._str_to_dt_quarter(self.start_date) <= self._str_to_dt_data(date) < self._str_to_dt_quarter(self.end_date):
            return True

        return False
    
    def add_analysis(self, datapoint):
        print("hash", datapoint['hash'])
        if datapoint['category'] == "commit":
            self.num_commits += 1 
            
            if datapoint['author'] not in old_committers:
                self.new_committers += 1
                
            old_committers.add(datapoint['author'])
            
                
        if datapoint['category'] == "issue":
            self.num_issues += 1 
            
            if datapoint['author'] not in old_issue_subs:
                self.new_issue_subs += 1
                
            old_issue_subs.add(datapoint['author'])
            
                
        if datapoint['category'] == "pull_request":
            self.num_pullrequests += 1 
            
            if datapoint['author'] not in old_pr_subs:
                self.new_pr_subs += 1
                
            old_pr_subs.add(datapoint['author'])
    
    @staticmethod
    def _str_to_dt_data(date):
        
        datetimestr =  datetime.datetime.strptime(date, "%a %b %d %H:%M:%S %Y %z").strftime("%Y-%m-%d")
        datetimeobj = datetime.datetime.strptime(datetimestr, "%Y-%m-%d")
        return datetimeobj
        
    @staticmethod
    def _str_to_dt_quarter(date):
        
        datetimeobj =  datetime.datetime.strptime(date, "%Y-%m-%d")
        return datetimeobj
    
    def __str__(self):
        return str(self.number) + " " + str(self.year)

In [ ]:
clean_line_dict = {}

In [19]:
class CleanJson():   
    
    def __init__(self, path_to_file):
        
        self.clean_data  = pd.DataFrame(columns=['repo'
                                                'hash,
                                                'category',
                                                'commit',
                                                'author',
                                                'author_date',
                                                'commit_date',
files_no'])
        
        with open(path_to_file, 'r') as raw_data:
            for line in raw_data:
                line = json.loads(line)

#                 print(line)
                clean_line = dict()
                if line['category'] == "commit":
                    clean_line = self._clean_commit(line)
                    print(type(clean_line))

                elif line['category'] == "issue":
                    clean_line = self._clean_issue(line)

                elif line['category'] == "pull_request":
                    clean_line = self._clean_pr(line)

                self.clean_data[line['category']].append(clean_line)
#                 print(self.clean_data[line['category']])
                    
    def _clean_commit(line):
            repo_name = line['origin']
            line_data = line['data']
            cleaned_line = {
                'repo': repo_name,
                'hash': line_data['commit'],
                'category': "commit",
                'commit': line_data['Commit'],
                'author': line_data['Author'],
                'author_date': line_data['AuthorDate'],
                'commit_date': line_data['CommitDate'],
                'files_no': len(line_data['files'])
            }

            actions = 0

            for file in line_data['files']:
                if 'action' in file:
                    actions += 1
                    cleaned_line['files_action'] = actions
                    cleaned_line['merge'] = 'Merge' in line_data
            return cleaned_line

    @staticmethod
    def _clean_issue(line):
        repo_name = line['origin']
        line_data = line['data']
        cleaned_line ={
            'repo': repo_name,
            'hash': line_data['id'],
            'category': "issue",
            'author': line_data['user']['login'],
            'created_date': line_data['created_at'],
            'current_status': line_data['state']   
        }

        return cleaned_line

    @staticmethod
    def _clean_pr(line):
        repo_name = line['origin']
        line_data = line['data']
        cleaned_line ={
            'repo': repo_name,
            'hash': line_data['id'],
            'category': "pull_request",
            'author': line_data['user']['login'],
            'created_date': line_data['created_at'],
            'current_status': line_data['state']   
        }

        return cleaned_line
    

In [20]:
year_list = [x for x in range(start_year, end_year + 1)]
quar_list = [Quarter(num, year)  for year in year_list for num in range(1, 5)]

In [21]:
clean_data = CleanJson('./Chaoss_19_Microtasks/github_data.json')

KeyError: 'issue'

In [127]:
for category in ("commit", "issue", "pull_request"):
    print(category)
    print(clean_data)
    data = clean_data.clean_data[category]
    print(type(data))
    for quarter in quar_list:
        
        for data_point in data:
            print(type(data_point))
            print(data_point)
            if quarter.is_includes_data(data_point["created_date"]):
                quarter.add_analysis(data_point)


commit
<class 'list'>
<class 'dict'>
{'repo': 'https://github.com/Polaris000/practise', 'hash': 'a6898dbd2f66e5c6705bc434fb03ab65787040b5', 'category': 'commit', 'commit': 'Aniruddha Karajgi <31214064+Polaris000@users.noreply.github.com>', 'author': 'Aniruddha Karajgi <31214064+Polaris000@users.noreply.github.com>', 'created_date': 'Wed Sep 13 11:43:05 2017 +0530', 'files_no': 1, 'files_action': 1, 'merge': False}
<class 'dict'>
{'repo': 'https://github.com/Polaris000/practise', 'hash': '370065bf7689f116332b0c771e26e037f1c29d8b', 'category': 'commit', 'commit': 'GitHub <noreply@github.com>', 'author': 'Aniruddha Karajgi <31214064+Polaris000@users.noreply.github.com>', 'created_date': 'Wed Sep 13 11:43:43 2017 +0530', 'files_no': 1, 'files_action': 1, 'merge': False}
<class 'dict'>
{'repo': 'https://github.com/Polaris000/practise', 'hash': 'de9275c5247083d748937cf977402f7e064c16a0', 'category': 'commit', 'commit': 'GitHub <noreply@github.com>', 'author': 'Aniruddha Karajgi <31214064+Pol

# Analysis



## Number of commits, pull requests and issues per quadrant

In [122]:
for q in quar_list:
    print(q, q.num_commits)
    
for i in old_committers:
    print(i)

1 2017 0
2 2017 9
3 2017 9
4 2017 45
1 2018 1
2 2018 8
3 2018 6
4 2018 0
Aniruddha Karajgi <akarajgi0@gmail.com>
sangeeta karajgi <akarajgi0@gmail.com>
Naman Kumar Garg <31842804+Naman-Garg-06@users.noreply.github.com>
Aniruddha Karajgi <31214064+Polaris000@users.noreply.github.com>


## Number of new committers, new issue submitters and pull request creators

In [123]:
for q in quar_list:
    print(q, q.new_committers)

1 2017 0
2 2017 2
3 2017 0
4 2017 2
1 2018 0
2 2018 0
3 2018 0
4 2018 0


# Viewing data as a csv file

## Writing the cleaned data to a csv

In [187]:
def write_to_csv(file_path):
    with open(file_path, 'w', ) as csvfile:
        csv_writer = csv.writer(csvfile, delimiter=',')
        
        file_headers = ["Quarter(Num)", "Quarter(Year)", "Num_Commits", "Num_Issues", "Num_PRs", "Num_new_commits", "Num_new_issues", "Num_new_prs"]
        csv_writer.writerow(file_headers)
        
        for quar in quar_list:
            row = [str(quar.number),       \
                 str(quar.year),         \
                 str(quar.num_commits) ,     \
                 str(quar.num_issues)  ,    \
                 str(quar.num_pullrequests), \
                 str(quar.new_committers),  \
                 str(quar.new_issue_subs), \
                 str(quar.new_pr_subs)       
                  ]
            csv_writer.writerow(x for x in row)
            

In [188]:
write_to_csv("./practise.csv")

## Displaying a table based on the csv file

In [189]:
def create_table(file_path):
    with open(file_path, 'r', ) as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=',')
        
        
        
        for row in csv_reader:
            for field in row:
                print("%-10s" %field, end="\t")
            print()
            
        
        
        

In [190]:
create_table('./practise.csv')

Quarter(Num)	Quarter(Year)	Num_Commits	Num_Issues	Num_PRs   	Num_new_commits	Num_new_issues	Num_new_prs	
1         	2017      	0         	0         	0         	0         	0         	0         	
2         	2017      	9         	0         	0         	0         	0         	0         	
3         	2017      	9         	0         	0         	0         	0         	0         	
4         	2017      	45        	0         	0         	0         	0         	0         	
1         	2018      	1         	0         	0         	0         	0         	0         	
2         	2018      	8         	0         	0         	0         	0         	0         	
3         	2018      	6         	0         	0         	0         	0         	0         	
4         	2018      	0         	0         	0         	0         	0         	0         	


In [22]:
a = pd.DataFrame(columns=['a', 'b', 'c'])

In [42]:
b = {'b': 5, 'c':  10, 'a': None}

In [43]:
a.add(b)

,a,b,c


In [44]:
a

,a,b,c
